In [ ]:
import ROOT as root
import numpy as np
import pandas as pd
import math
import uproot
import os
import sys
import root_pandas as rp
import analysis_variables as a_v
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn import metrics
import torch
from torch.autograd import Function
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import qiskit
from qiskit import transpile, assemble
from qiskit.visualization import *
from tqdm.notebook import tqdm

In [ ]:
# Create a “Quantum Class” with Qiskit
class QuantumCircuit:
    """ 
    This class provides a simple interface for interaction 
    with the quantum circuit 
    """
    def __init__(self, n_qubits, backend, shots):
        # --- Circuit definition ---
        self._circuit = qiskit.QuantumCircuit(n_qubits)
        all_qubits = [i for i in range(n_qubits)]
        self.theta = qiskit.circuit.Parameter('theta')
        self._circuit.h(all_qubits)
        self._circuit.barrier()
        self._circuit.ry(self.theta, all_qubits)
        self._circuit.measure_all()
        # ---------------------------
        self.backend = backend
        self.shots = shots
    def run(self, thetas):
        t_qc = transpile(self._circuit,
                         self.backend)
        qobj = assemble(t_qc,
                        shots=self.shots,
                        parameter_binds = [{self.theta: theta} for theta in thetas])
        job = self.backend.run(qobj)
        result = job.result().get_counts()
        counts = np.array(list(result.values()))
        states = np.array(list(result.keys())).astype(float)
        # Compute probabilities for each state
        probabilities = counts / self.shots
        # Get state expectation
        expectation = np.sum(states * probabilities)
        return np.array([expectation])
    
# test the implementation
simulator = qiskit.Aer.get_backend('aer_simulator')
circuit = QuantumCircuit(1, simulator, 100)
print('Expected value for rotation pi {}'.format(circuit.run([np.pi])[0]))
circuit._circuit.draw('mpl')

In [ ]:
# Create a “Quantum-Classical Class” with PyTorch
class HybridFunction(Function):
    """ Hybrid quantum - classical function definition """
    @staticmethod
    def forward(ctx, input, quantum_circuit, shift):
        """ Forward pass computation """
        ctx.shift = shift
        ctx.quantum_circuit = quantum_circuit
        expectation_z = ctx.quantum_circuit.run(input[0].tolist())
        result = torch.tensor([expectation_z])
        ctx.save_for_backward(input, result)
        return result
    @staticmethod
    def backward(ctx, grad_output):
        """ Backward pass computation """
        input, expectation_z = ctx.saved_tensors
        input_list = np.array(input.tolist())
        shift_right = input_list + np.ones(input_list.shape) * ctx.shift
        shift_left = input_list - np.ones(input_list.shape) * ctx.shift
        gradients = []
        for i in range(len(input_list)):
            expectation_right = ctx.quantum_circuit.run(shift_right[i])
            expectation_left  = ctx.quantum_circuit.run(shift_left[i])
            gradient = torch.tensor([expectation_right]) - torch.tensor([expectation_left])
            gradients.append(gradient)
        gradients = np.array([gradients]).T
        return torch.tensor([gradients]).float() * grad_output.float(), None, None
class Hybrid(nn.Module):
    """ Hybrid quantum - classical layer definition """
    def __init__(self, backend, shots, shift):
        super(Hybrid, self).__init__()
        self.quantum_circuit = QuantumCircuit(1, backend, shots)
        self.shift = shift
    def forward(self, input):
        return HybridFunction.apply(input, self.quantum_circuit, self.shift)

In [ ]:
apply_on_files = 1 # 0 or 1
do_grid_search = 0

cs_vars = ['','','','....']

target_var = ['Signal']
contSupp_var = ['contSupp_BB_lep']

train_path = a_v.input_dir + 'BDT_contSupp_BB_train_2023-12-06_RW_2.parq'
test_path = a_v.input_dir + 'BDT_contSupp_BB_test_2023-12-06.parq'

train = pd.read_parquet(train_path, engine='pyarrow')
test = pd.read_parquet(test_path, engine='pyarrow')

In [ ]:
x_train = train[cs_vars]
x_train_weights = train.contReweight
y_train = train[target_var]
x_test = test[cs_vars]
y_test = test[target_var]

In [ ]:
x_train = x_train.to_numpy()
y_train_2 = y_train.to_numpy()
y_train = []
for i in y_train_2:
    y_train.append(int(i))
x_test = x_test.to_numpy()
y_test_2 = y_test.to_numpy()
y_test = []
for i in y_test_2:
    y_test.append(int(i))

x_train = torch.tensor(x_train,dtype=torch.float32)
y_train = torch.tensor(y_train,dtype=torch.float32).type(torch.LongTensor)
x_test = torch.tensor(x_test,dtype=torch.float32)
y_test = torch.tensor(y_test,dtype=torch.float32).type(torch.LongTensor)

dataset_train = torch.utils.data.TensorDataset(x_train, y_train)
dataset_train.data = dataset_train[:][0]
dataset_train.targets = dataset_train[:][1]
dataset_test = torch.utils.data.TensorDataset(x_test, y_test)
dataset_test.data = dataset_test[:][0]
dataset_test.targets = dataset_test[:][1]

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=1, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=1, shuffle=False)

In [ ]:
# Creating the Hybrid Neural Network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.dropout = nn.Dropout2d()
        self.fc1 = nn.Linear(9, 4)
        self.fc2 = nn.Linear(4, 1)
        self.hybrid = Hybrid(qiskit.Aer.get_backend('aer_simulator'), 100, np.pi / 2)
    def forward(self, x):
        #x = F.relu(self.conv1(x))
        #x = F.max_pool2d(x, 2)
        #x = F.relu(self.conv2(x))
        #x = F.max_pool2d(x, 2)
        #x = self.dropout(x)
        #x = x.view(1, -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.hybrid(x)
        return torch.cat((x, 1 - x), -1)

In [ ]:
# Training the Network
model = Net()
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_func = nn.NLLLoss()
epochs = 20
loss_list = []
model.train()
for epoch in tqdm(range(epochs)):
    total_loss = []
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        # Forward pass
        output = model(data)
        # Calculating loss
        loss = loss_func(output, target)
        # Backward pass
        loss.backward()
        # Optimize the weights
        optimizer.step()
        total_loss.append(loss.item())
    loss_list.append(sum(total_loss)/len(total_loss))
    print('Training [{:.0f}%]\tLoss: {:.4f}'.format(
        100. * (epoch + 1) / epochs, loss_list[-1]))